# Imputer
- github colab : https://homl.info/colab3
- Data에 아예 input이 누락되어 있는 곳이 있을 수 있다
- 이러한 data에 대응하는 방법에는 다음이 있을 수 있다
    - 특정한 값(예 : 0, median, mean)으로 채운다
    - 누락된 data가 있는 row를 제외한다
    - 누락된 data가 있는 column을 제외한다
- 특정한 값으로 채우는 역할을 해주는 걸 Imputer라고 한다

In [174]:
import matplotlib
import numpy
import pathlib
import pandas
import sklearn
import sklearn.impute
import sklearn.model_selection
import sklearn.preprocessing
import tarfile
import urllib

def ch2_load_housing_data():
    tarball_path = pathlib.Path("datasets/housing.tgz")
    if not tarball_path.is_file():
        pathlib.Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://github.com/ageron/data/raw/main/housing.tgz"
        urllib.request.urlretrieve(url, tarball_path)
        with tarfile.open(tarball_path) as housing_tarball:
            housing_tarball.extractall(path="datasets")
    return pandas.read_csv(pathlib.Path("datasets/housing/housing.csv"))

def matplotlib_to_imagefile(output_dir, filename, imgext="png", tight_layout=True, resolution=300):
    path = output_dir / f"{filename}.{imgext}"
    if tight_layout:
        matplotlib.pyplot.tight_layout()
    matplotlib.pyplot.savefig(path, format=imgext, dpi=resolution)
    
def stratified_sampling_income_category(input_dataframe):
    input_dataframe = input_dataframe.copy()
    input_dataframe["income_cat"] = pandas.cut(input_dataframe["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., numpy.inf],
                               labels=[1, 2, 3, 4, 5])
    s_train, s_test = sklearn.model_selection.train_test_split(input_dataframe, test_size = 0.2, stratify = input_dataframe['income_cat'], random_state = 42)
    s_train.drop('income_cat', axis=1, inplace=True)
    s_test.drop('income_cat', axis=1, inplace=True)
    
    return s_train, s_test
    
# 저장할 디렉토리 설정
output_dir = pathlib.Path() / "images" / "end_to_end_project"
output_dir.mkdir(parents=True, exist_ok=True)
print(f'output_dir : {output_dir}')

input_dataframe = ch2_load_housing_data()
train, test = stratified_sampling_income_category(input_dataframe)

output_dir : images\end_to_end_project


In [175]:
#
# 우리가 예측하고자 하는 값을 label이라고 하고 label을 예측하기 위한 근거/데이터를 predictor라고 한다
# 바꿔 말해 우리는 predictor를 통해 label을 예측할 수 있는 프로그램을 만들고자 하는 것이다
#
label = train['median_house_value'].copy()
predictor = train.drop('median_house_value', axis = 1)  # 0 = drop row / 1 = drop column

In [176]:
#
# recap : total_bedrooms에 빠진 데이터가 있다
#
input_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [177]:
# pandas.DataFrame.isnull 은 isna의 alias이다. null/NaN/None이면 True이고 아니면 False인 DataFrame을 리턴한다 (크기는 같다)
predictor.isnull()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
13096,False,False,False,False,False,False,False,False,False
14973,False,False,False,False,False,False,False,False,False
3785,False,False,False,False,False,False,False,False,False
14689,False,False,False,False,False,False,False,False,False
20507,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
14207,False,False,False,False,False,False,False,False,False
13105,False,False,False,False,False,False,False,False,False
19301,False,False,False,False,False,False,False,False,False
19121,False,False,False,False,False,False,False,False,False


In [178]:
# pandas.DataFrame.any는 True인게 하나라도 있는지에 대한걸 조사해 준다
# axis=1 이면 row중에 하나라도 True가 있는지 여부다
predictor.isnull().any(axis=1).sort_values()

13096    False
8495     False
16803    False
4537     False
4606     False
         ...  
4504      True
7018      True
6857      True
13498     True
4091      True
Length: 16512, dtype: bool

In [179]:
#
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html
# pandas.DataFrame.loc 은 기본적으로 label-based이다
# 하지만 boolean array를 받을 수도 있으며 이 경우에는 true에 해당하는 row만을 리턴한다
#
bool_arr = predictor.isnull().any(axis=1)
predictor.loc[bool_arr]

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
14452,-120.67,40.50,15.0,5343.0,NaN,2503.0,902.0,3.5962,INLAND
18217,-117.96,34.03,35.0,2093.0,NaN,1755.0,403.0,3.4115,<1H OCEAN
11889,-118.05,34.04,33.0,1348.0,NaN,1098.0,257.0,4.2917,<1H OCEAN
20325,-118.88,34.17,15.0,4260.0,NaN,1701.0,669.0,5.1033,<1H OCEAN
14360,-117.87,33.62,8.0,1266.0,NaN,375.0,183.0,9.8020,<1H OCEAN
...,...,...,...,...,...,...,...,...,...
2348,-122.70,38.35,14.0,2313.0,NaN,954.0,397.0,3.7813,<1H OCEAN
366,-122.50,37.75,44.0,1819.0,NaN,1137.0,354.0,3.4919,NEAR OCEAN
18241,-121.44,38.54,39.0,2855.0,NaN,1217.0,562.0,3.2404,INLAND
18493,-116.21,33.75,22.0,894.0,NaN,830.0,202.0,3.0673,INLAND


- 이와 같이 NaN/Null/None 이 있는지 확인을 했다면, 이것들을 어떻게 할것인지에 대해 생각해 봐야 함

In [180]:
# Option 1 : null이 있는 row들을 삭제한다
pred_drop_na = predictor.dropna(subset=['total_bedrooms'])

In [181]:
# Option 2 : null을 포함하는 column을 제거한다
pred_drop_col = predictor.drop('total_bedrooms', axis = 1)

In [182]:
# Option 3 : null을 다른 뭔가로 채워 넣는다 <- 여기서는 이걸로
median = predictor['total_bedrooms'].median()
print(f"predictor['total_bedrooms'] = {median}")
predictor['total_bedrooms'].fillna(median, inplace = True)

predictor['total_bedrooms'] = 434.0


- scikit-learn에서는 SimpleImputer라고 해서 저런걸 해주는 class 가 존재함

In [183]:
imputer = sklearn.impute.SimpleImputer(strategy='median')

In [184]:
#
# 위를 사용하려면 일단 type이 number인것들을 추려내서 적용해야 함. 다음과 같이 할 수 있다
# fit은 transform에 필요한 계산을 한다
# transform은 실제 대체를 한다
#
predictor_numtype = predictor.select_dtypes(include=[numpy.number])
imputer.fit(predictor_numtype)

SimpleImputer(strategy='median')

In [185]:
# imputer가 계산한게 DataFrame의 median과 일치하는지 확인시켜주는 코드
print(imputer.statistics_)
print(predictor_numtype.median().values)

[-118.51     34.26     29.     2125.      434.     1167.      408.
    3.5385]
[-118.51     34.26     29.     2125.      434.     1167.      408.
    3.5385]


In [186]:
# 실제 대체가 이루어 지는 곳은 transform
X = imputer.transform(predictor_numtype)
print(type(X)) # 그런데 impute는 numpy.ndarray를 리턴함

# 따라서 이걸 쓰기 편한 DataFrame으로 변환한다
predictor_numtype = pandas.DataFrame(X, columns = predictor_numtype.columns, index = predictor_numtype.index)
predictor_numtype.info() # 이제 모두 Non-Null이 된것을 볼 수 있다

<class 'numpy.ndarray'>
<class 'pandas.core.frame.DataFrame'>
Int64Index: 16512 entries, 13096 to 19888
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           16512 non-null  float64
 1   latitude            16512 non-null  float64
 2   housing_median_age  16512 non-null  float64
 3   total_rooms         16512 non-null  float64
 4   total_bedrooms      16512 non-null  float64
 5   population          16512 non-null  float64
 6   households          16512 non-null  float64
 7   median_income       16512 non-null  float64
dtypes: float64(8)
memory usage: 1.1 MB


# Category Encoding

- ocean_proximity 같은 경우 string 으로 되어 있다
- 기본적으로 이런 데이터를 category라고 하는데, 원활한 machine learning을 위해서는 수치화가 필요하다
- low, below average, average, above average, high 뭐 이런식으로 연관이 되어 있는 거면 각각 1,2,3,4,5 이런식으로 매핑할수도 있다
- 카테고리는 있는데 이게 linear relationship을 갖고 있지 않다면 그냥 유/무를 bit로 표현할수도 있다. one-hot encoding이라고도 한다
- 여기서는 one-hot encoding을 사용해 본다

In [187]:
predictor['ocean_proximity'].value_counts()

<1H OCEAN     7274
INLAND        5301
NEAR OCEAN    2089
NEAR BAY      1846
ISLAND           2
Name: ocean_proximity, dtype: int64

In [188]:
# Series가 아니라 DataFrame으로 return 시키기 위해 [[]] 를 사용
predictor_category = predictor[['ocean_proximity']]
print(type(predictor_category))

<class 'pandas.core.frame.DataFrame'>


In [189]:
#
# Ordinal encoding은 말 그대로 category를 숫자에 대응 시키는 방법이다
# 여기에서는 one-hot encoding 을 주로 다루겠지만 일단 ordinal encoding은 이렇게 사용한다는 것을 남기기 위해 코드를 써둠
#
ordinal = sklearn.preprocessing.OrdinalEncoder()
predictor_category_ordinal = ordinal.fit_transform(predictor_category)
predictor_category_ordinal

array([[3.],
       [0.],
       [1.],
       ...,
       [4.],
       [0.],
       [4.]])

In [190]:
onehot = sklearn.preprocessing.OneHotEncoder()
predictor_category_1hot = onehot.fit_transform(predictor_category)
# return type이 dense matrix가 아닌 sparse matrix임
print(type(predictor_category_1hot))  
# dense matrix로 변환 하려면 이렇게 하면 된다
predictor_category_1hot.toarray()

<class 'scipy.sparse._csr.csr_matrix'>


array([[0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.]])

In [191]:
# 처음부터 dense matrix를 만드는게 목표인 경우 (현재 그렇다) 다음과 같이 할 수도 있다
onehot = sklearn.preprocessing.OneHotEncoder( sparse_output = False )
predictor_category_1hot = onehot.fit_transform(predictor_category)
predictor_category_1hot

array([[0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.]])

In [192]:
#
# sklearn.preprocessing.OneHotEncoder의 좋은 점은 한 데이터에 fit 해두면 다른 데이터에도 동일한 형태로 transform해준다는 것이다
# pandas.get_dummies는 OneHotEncoder와 비슷한 일을 해 주는데 대신 그때그때 있는 데이터로 category to number transformation을 하지
# 이전의 transformation을 갖고와서 그대로 쓰지 않기 때문에 매번 category화 한 결과가 달라질 수 있다
# 아래의 예를 보면 대충 이해가 될 듯. 말로 설명하기 힘드네

print(onehot.categories_)
print()

another_data = pandas.DataFrame({"ocean_proximity" : ["INLAND", "NEAR BAY"]})

print(pandas.get_dummies(another_data))
print()

onehot.handle_unknown = 'ignore'
another_data_result = onehot.transform(another_data)
print(another_data_result) # 이 결과를 보면 원래 갖고 있던 category to bit mapping 방법에 따라 mapping했다. 즉 빈칸 수가 같다

[array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
      dtype=object)]

   ocean_proximity_INLAND  ocean_proximity_NEAR BAY
0                       1                         0
1                       0                         1

[[0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]]


In [193]:
#
# DataFrame을 Scikit-Learn의 Estimator fit에 사용하면 estimator는 언제나 feature_names_in_ 에 column name을 남긴다
# 즉 이후에 다른 자료를 Fit할때에도 같은 column 구조를 이렇게 보고 사용하면 된다
#
print(onehot.feature_names_in_)
print()

#
# 반대로 fit한결과로 사용한 column name은 feature_names_out에 들어가 있다
# 나중에 DataFrame에 반영할때 이것을 사용하면 될 것이다
#
print(onehot.get_feature_names_out())

['ocean_proximity']

['ocean_proximity_<1H OCEAN' 'ocean_proximity_INLAND'
 'ocean_proximity_ISLAND' 'ocean_proximity_NEAR BAY'
 'ocean_proximity_NEAR OCEAN']
